# Logistics Performance Solution
### by Yuan Du, 12/18/2019
Code stored at google drive:https://drive.google.com/drive/folders/1JJUJXtppV_qNBTl9ittC5czPYvFdxJCe
## Tasks
Identify all the orders that are considered late depending on the Service Level Agreements (SLA) with our Logistics Provider.

+ [a. Import Libraries.](#a)<br>
+ [b. Import data.](#b)<br>
+ [c. Count SLA day, flatten the matrix.](#c)<br>
+ [d. Merge SLA with data](#d)<br>
+ [e. Convert date to GMT+8](#e)<br>
+ [f. Calculate gap](#f)<br>
+ [g. Output decision](#g)<br>
***
### <a id=a>a. Import Libraries</a>

In [ ]:
#Import libs
import numpy as np
import pandas as pd

### <a id=b>b. Import data</a>

In [ ]:
#Import Data
data = pd.read_csv("C:/Work/Project/Acclivity/delivery_orders_march.csv/delivery_orders_march.csv")
#split address to location
data["buy"] = data["buyeraddress"].apply(lambda x: x.split(" ")[-1].lower())
data["sell"] = data["selleraddress"].apply(lambda x: x.split(" ")[-1].lower())

In [ ]:
data.head()

In [ ]:
data.info()

### <a id=c>c. Count SLA day, flatten the matrix</a>

In [ ]:
#count SLA day, flatten the matrix
SLA = pd.DataFrame(
    data = [
            ["manila","manila",3],
            ["manila","luzon",5],
            ["manila","visayas",7],
            ["manila","mindanao",7],
            ["luzon","manila",5],
            ["luzon","luzon",5],
            ["luzon","visayas",7],
            ["luzon","mindanao",7],
            ["visayas","manila",7],
            ["visayas","luzon",7],
            ["visayas","visayas",7],
            ["visayas","mindanao",7],
            ["mindanao","manila",7],
            ["mindanao","luzon",7],
            ["mindanao","visayas",7],
            ["mindanao","mindanao",7]
           ],
    columns=["buy", "sell", "SLA"]
)

In [ ]:
SLA

### <a id=d>d. Count SLA day, flatten the matrix</a>

In [ ]:
#Merge
data = pd.merge(data, SLA, on = ["buy", "sell"])

In [ ]:
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,buy,sell,SLA
0,2215676524,2020-03-02,2020-03-05,1970-01-01,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,manila,manila,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",manila,manila,3
2,2220979489,2020-03-04,2020-03-06,1970-01-01,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",manila,manila,3
3,2221066352,2020-03-05,2020-03-07,1970-01-01,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",manila,manila,3
4,2222597288,2020-03-04,2020-03-07,1970-01-01,C-846 park&shop opens Valeda/Sunrise) CompuWar...,33 sta.mesa 1929 hawthorn Metro Manila,manila,manila,3


### <a id=e>e. Convert date to GMT+8 </a>

In [ ]:
#GMT+8
data[["pick", "1st_deliver_attempt", "2nd_deliver_attempt"]] += 8*60*60
#to date time
data["pick"] = pd.to_datetime(data["pick"],unit='s').dt.date
data["1st_deliver_attempt"] = pd.to_datetime(data["1st_deliver_attempt"],unit='s').dt.date
data["2nd_deliver_attempt"].fillna(0, inplace = True)#busday_count can't deal with na
data["2nd_deliver_attempt"] = pd.to_datetime(data["2nd_deliver_attempt"],unit='s').dt.date

In [ ]:
#count workday
holidays = ["2020-03-08", "2020-03-25", "2020-03-30", "2020-03-31"]

In [ ]:
#distinct pickup date
data["pick"].unique()

array([datetime.date(2020, 3, 2), datetime.date(2020, 3, 4),
       datetime.date(2020, 3, 5), datetime.date(2020, 3, 7),
       datetime.date(2020, 3, 6), datetime.date(2020, 3, 10),
       datetime.date(2020, 3, 19), datetime.date(2020, 3, 9),
       datetime.date(2020, 3, 11), datetime.date(2020, 3, 21),
       datetime.date(2020, 3, 13), datetime.date(2020, 3, 14),
       datetime.date(2020, 3, 12), datetime.date(2020, 3, 16),
       datetime.date(2020, 3, 20), datetime.date(2020, 3, 17),
       datetime.date(2020, 3, 18), datetime.date(2020, 3, 26),
       datetime.date(2020, 3, 23), datetime.date(2020, 4, 2),
       datetime.date(2020, 3, 27), datetime.date(2020, 3, 28),
       datetime.date(2020, 4, 3), datetime.date(2020, 4, 6),
       datetime.date(2020, 4, 4), datetime.date(2020, 3, 3),
       datetime.date(2020, 3, 24)], dtype=object)

In [ ]:
#distinct 1st_deliver_attempt date
data["1st_deliver_attempt"].unique()

In [ ]:
#check if there is pickup/1st_deliver_attempt date on sunday, if so needs to shift start date 1 day==>no
data["weekday_pick"]= pd.to_datetime(data["pick"]).dt.dayofweek
pick_sun=data[data["weekday_pick"]==6]
pick_sun.head()

### <a id=f>f. Calculate gap</a>

In [ ]:
#Calculate gap
data["frist_gap"] = np.busday_count(data["pick"], data["1st_deliver_attempt"], weekmask="1111110", holidays = holidays)
data["second_gap"] = np.busday_count(data["1st_deliver_attempt"], data["2nd_deliver_attempt"], weekmask="1111110", holidays = holidays)

### <a id=g>g. Output decision</a>

In [ ]:
#Output decision
data["is_late"] = (data["frist_gap"] > data["SLA"]) | (data["second_gap"] > 3)
data["is_late"] = data["is_late"].astype(int)

In [ ]:
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,buy,sell,SLA,frist_gap,second_gap,is_late
0,2215676524,2020-03-02,2020-03-05,1970-01-01,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,manila,manila,3,3,-15708,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",manila,manila,3,2,3,0
2,2220979489,2020-03-04,2020-03-06,1970-01-01,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",manila,manila,3,2,-15709,0
3,2221066352,2020-03-05,2020-03-07,1970-01-01,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",manila,manila,3,2,-15710,0
4,2222597288,2020-03-04,2020-03-07,1970-01-01,C-846 park&shop opens Valeda/Sunrise) CompuWar...,33 sta.mesa 1929 hawthorn Metro Manila,manila,manila,3,3,-15710,0


In [ ]:
#Final output
Output = data[["orderid","is_late"]]
Output.to_csv("C:/Work/Project/Acclivity/delivery_orders_march.csv/Output.csv", index = False)